In [1]:
# %%
import numpy as np
from scipy.linalg import svd
import sys
sys.path.append("../")
import scipy.io as sio
import os


In [2]:
import h5py
with h5py.File('pressure_data.mat', 'r') as file:
    # Load DATA_U and transpose it
    DATA = np.array(file['/DATA']).T  # Transpose the data
print("size of DATA: ", DATA.shape)

size of DATA:  (798, 295122)


In [4]:
# %%
# Load the MAT file

# Define the number of singular values/components to keep
k = 300  # for example, keep 100 components

# Perform SVD and keep only the first k of U, S, and VT
U_reduced, S_reduced, VT_reduced = svd(DATA, full_matrices=False)
U_reduced = U_reduced[:, :k]
S_reduced = S_reduced[:k]
VT_reduced = VT_reduced[:k, :]

# Reconstruct the reduced data matrix
DATA_reduced = np.dot(U_reduced, np.diag(S_reduced))

# Now you can slice the DATA as before
X = DATA_reduced[:-1]  # All rows except the last
Y = DATA_reduced[1:]   # All rows except the first

print("size of DATA reduced: ", DATA_reduced.shape)
print("size of X: ", X.shape)
# Clean up
# del temp

len_all = X.shape[0]
data_x_train = X[:int(0.7*len_all),:]
data_x_valid = X[int(0.7*len_all)+1:,:]

data_y_train = Y[:int(0.7*len_all),:]
data_y_valid = Y[int(0.7*len_all)+1:,:]

data_train = [data_x_train, data_y_train]
data_valid = [data_x_valid, data_y_valid]

size of DATA reduced:  (798, 300)
size of X:  (797, 300)


In [4]:
# %%
from solver_resdmd import KoopmanNN, KoopmanSolver

# basis_function = PsiNN(layer_sizes=[100,100,100], n_psi_train=n_psi_train)
# N_dict = n_psi_train + 3
# # Dynamically import the solver module based on solver_index
# filename = f'solver_{solver_index}_outputs_{N_dict}basis.mat'
# solver_module = __import__(f"solver_{solver_index}", fromlist=['KoopmanDLSolver'])
# KoopmanDLSolver = getattr(solver_module, 'KoopmanDLSolver')
n_psi=149
basis_function = KoopmanNN(layer_sizes=[150,150,150], n_psi_train=n_psi)

# Using the dynamically imported solver
solver = KoopmanSolver(dic=basis_function, 
                            target_dim=np.shape(data_train)[-1], 
                            reg=0.1)
solver.build(data_train=data_train, 
                data_valid=data_valid, 
                epochs=300, 
                batch_size=500, 
                lr=1e-4, 
                log_interval=10, 
                lr_decay_factor=.8)

# Results from solver
evalues = solver.eigenvalues
efuns = solver.eigenfunctions(X)
N_dict = np.shape(evalues)[0]    
Psi_X = solver.get_Psi_X().numpy()
Psi_Y = solver.get_Psi_Y().numpy()
Koopman_matrix_K = solver.K.numpy()
# kpm_modes = solver.compute_mode()
kpm_modes = np.dot(VT_reduced.T, solver.compute_mode())

# Prepare data to save
outputs = {
    'evalues': evalues,
    'efuns': efuns,
    'N_dict': N_dict,
    'Psi_X': Psi_X,
    'Psi_Y': Psi_Y,
    'K': Koopman_matrix_K,
    'kpm_modes': kpm_modes,
}

folder_path = 'data/turbulence_data'  # Adjust the path as needed if you want a different location
os.makedirs(folder_path, exist_ok=True)
# Save the outputs to a .mat file named according to the number of basis functions, inside the "data" folder
file_path = os.path.join(folder_path, f'turbulence_resdmd_{N_dict}basis.mat')
sio.savemat(file_path, outputs)
print(f'Saved outputs for n_psi_train={n_psi} to {file_path}')






Outer Epoch 1/300
Epoch 1/2
2/2 [==============================] - 2s 602ms/step - loss: 289.6801 - val_loss: 25859.0352
Epoch 2/2
2/2 [==============================] - 0s 47ms/step - loss: 289.6801 - val_loss: 25859.0312
Outer Epoch 2/300
Epoch 1/2
2/2 [==============================] - 0s 60ms/step - loss: 289.6800 - val_loss: 25859.0332
Epoch 2/2
2/2 [==============================] - 0s 35ms/step - loss: 289.6801 - val_loss: 25859.0312
Outer Epoch 3/300
Epoch 1/2
2/2 [==============================] - 0s 69ms/step - loss: 289.6800 - val_loss: 25859.0352
Epoch 2/2
2/2 [==============================] - 0s 35ms/step - loss: 289.6800 - val_loss: 25859.0391
Outer Epoch 4/300
Epoch 1/2
2/2 [==============================] - 0s 67ms/step - loss: 289.6800 - val_loss: 25859.0352
Epoch 2/2
2/2 [==============================] - 0s 40ms/step - loss: 289.6801 - val_loss: 25859.0352
Outer Epoch 5/300
Epoch 1/2
2/2 [==============================] - 0s 64ms/step - loss: 289.6800 - val_loss: 25